<span style="color:red"><b><<<<<<< local</b></span>

In [1]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession, functions as F
import os
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/10/06 13:50:46 WARN Utils: Your hostname, DESKTOP-Q5SP5SI resolves to a loopback address: 127.0.1.1; using 172.20.36.110 instead (on interface eth0)
24/10/06 13:50:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/06 13:50:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/06 13:50:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [8]:
# Reading Parquet Data
sdf_3000 = spark.read.parquet('../data/raw/work_3000.parquet')

In [9]:
# now, we want to ensure everything has consistent casing to make our lives easier
consistent_col_casing = [F.col(col_name).alias(col_name.lower()) for col_name in sdf_3000.columns]
sdf_3000 = sdf_3000.select(*consistent_col_casing)

# this will be used in the cell below when reading in
sdf_schema = sdf_3000.schema
sdf_schema

StructType([StructField('url', StringType(), True), StructField('postcode', StringType(), True), StructField('suburb', StringType(), True), StructField('name', StringType(), True), StructField('cost_text', StringType(), True), StructField('beds', StringType(), True), StructField('baths', StringType(), True), StructField('parking', StringType(), True), StructField('property_type', StringType(), True)])

In [10]:
# Change data type
sdf = spark.read.schema(sdf_schema).parquet('../data/raw/*')

In [11]:
# Create new parquet of raw data
sdf \
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet('../data/scraped/property_data.parquet')

24/09/18 16:08:15 WARN TaskSetManager: Stage 5 contains a task of very large size (3811 KiB). The maximum recommended task size is 1000 KiB.


In [12]:
sdf = spark.read.parquet('../data/scraped/property_data.parquet')
print((sdf.count(), len(sdf.columns)))

(16003, 9)


In [13]:
sdf.sort('postcode')

url,postcode,suburb,name,cost_text,beds,baths,parking,property_type
https://www.domai...,3000,melbourne,1406/22 Jane Bell...,$795 include bill...,2,1,1,Apartment / Unit ...
https://www.domai...,3000,melbourne,2514/23 Mackenzie...,$720 per week *Pa...,2,1,0,Apartment / Unit ...
https://www.domai...,3000,melbourne,1202/601 Little C...,"$620 per week, $2...",2,1,0,Apartment / Unit ...
https://www.domai...,3000,melbourne,4302/120 A'Becket...,$1200 **INSPECTIO...,3,2,0,Apartment / Unit ...
https://www.domai...,3000,melbourne,4207/371 Little L...,$720 per week opp...,2,1,0,Apartment / Unit ...
https://www.domai...,3000,melbourne,4007/135 A'Becket...,$600 **INSPECTION...,1,1,0,Apartment / Unit ...
https://www.domai...,3000,melbourne,1302/279-283 La T...,$650 and Fully Fu...,2,1,0,Apartment / Unit ...
https://www.domai...,3000,melbourne,5809/442 ELIZABET...,$850 Per Week ( F...,2,1,0,Apartment / Unit ...
https://www.domai...,3000,melbourne,913/22-24 Jane Be...,$520 and Fully Fu...,1,1,1,Apartment / Unit ...
https://www.domai...,3000,melbourne,1813/135 A'Becket...,$550 **INSPECTION...,1,1,0,Apartment / Unit ...


In [15]:
sdf = sdf.dropDuplicates(['name'])

<span style="color:red"><b>=======</b></span>

In [17]:
print((sdf.count(), len(sdf.columns)))

(12101, 9)


<span style="color:red"><b>>>>>>>> remote</b></span>